In [1]:
# Run in a Jupyter cell
from pathlib import Path
ROOT = Path('plant')          # <-- your current root (change if different)
CROP_CLASS_DIR = Path('data/crops_for_classification')  # created below
MODEL_DIR = Path('models'); MODEL_DIR.mkdir(exist_ok=True, parents=True)

In [2]:
import os
print(os.getcwd())

C:\Users\Kashish Kharb\plant


In [3]:
pip install tensorflow==2.18.0 numpy==1.26.4

Note: you may need to restart the kernel to use updated packages.


In [4]:
# list crops and disease folders + counts
import os, random
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

root = Path(r"C:\Users\Kashish Kharb\plant")
for crop in sorted([p for p in root.iterdir() if p.is_dir()]):
    disease_folders = [d for d in crop.iterdir() if d.is_dir()]
    counts = {d.name: len(list(d.glob('*.jpg'))) for d in disease_folders}
    print(f"{crop.name}: {counts}")

# show a random sample image
crops = [p for p in root.iterdir() if p.is_dir()]
if crops:
    crop = random.choice(crops)
    diseases = [p for p in crop.iterdir() if p.is_dir()]
    if diseases:
        disease = random.choice(diseases)
        img_files = list(disease.glob('*.jpg')) + list(disease.glob('*.jpeg')) + list(disease.glob('*.png'))
        if img_files:
            img_path = random.choice(img_files)
            img = Image.open(img_path)
            plt.figure(figsize=(4,4))
            plt.imshow(img)
            plt.title(f"{crop.name}/{disease.name}")
            plt.axis('off')
            plt.show()
        else:
            print(f"No images found in {disease}")
    else:
        print(f"No disease folders found in {crop}")
else:
    print("No crop folders found.")

.ipynb_checkpoints: {}
corn: {'corn_common_rust': 1192, 'corn_gray_leaf_spot': 513, 'corn_healthy': 1162, 'corn_northern_leaf_blight': 985}
data: {'crops_for_classification': 0}
models: {}
potato: {'potato_early_blight': 1000, 'potato_healthy': 152, 'potato_late_blight': 1000}
rice: {'rice_brown_spot': 613, 'rice_healthy': 1488, 'rice_leaf_blast': 977, 'rice_neck_blast': 1000}
saved_models: {}
src: {'.ipynb_checkpoints': 0, '__pycache__': 0}
sugarcane: {'bacterial blight': 100, 'healthy': 100, 'red rot': 100}
wheat: {'wheat_brown_rust': 902, 'wheat_healthy': 1116, 'wheat_yellow_rust': 924}
No disease folders found in C:\Users\Kashish Kharb\plant\models


In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import joblib
from pathlib import Path

# === Paths ===
BASE_DIR = Path("data/train")
MODEL_DIR = Path("models")
MODEL_DIR.mkdir(exist_ok=True)

# === Data generator ===
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    BASE_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    BASE_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# === Model ===
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# === Train ===
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5
)

# === Save model & label map ===
model.save(MODEL_DIR / "crop_classifier.keras")
joblib.dump(train_gen.class_indices, MODEL_DIR / "crop_label_map.joblib")

print("✅ Model and label map saved successfully!")


Found 10661 images belonging to 5 classes.
Found 2663 images belonging to 5 classes.
Epoch 1/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 673s 2s/step - accuracy: 0.8760 - loss: 0.3840 - val_accuracy: 0.9542 - val_loss: 0.1192
Epoch 2/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 505s 2s/step - accuracy: 0.9586 - loss: 0.1333 - val_accuracy: 0.9373 - val_loss: 0.1746
Epoch 3/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 511s 2s/step - accuracy: 0.9689 - loss: 0.0960 - val_accuracy: 0.9670 - val_loss: 0.0833
Epoch 4/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 512s 2s/step - accuracy: 0.9781 - loss: 0.0717 - val_accuracy: 0.9493 - val_loss: 0.1532
Epoch 5/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 512s 2s/step - accuracy: 0.9768 - loss: 0.0660 - val_accuracy: 0.9670 - val_loss: 0.0850
✅ Model and label map saved successfully!


In [5]:
# # create crop-level folders by copying (no change to original)
# import shutil
# from pathlib import Path

# ROOT = Path(r"C:\Users\Kashish Kharb\plant")
# OUT = Path('data/crops_for_classification')
# OUT.mkdir(parents=True, exist_ok=True)

# for crop in [p for p in ROOT.iterdir() if p.is_dir()]:
#     target = OUT/crop.name
#     target.mkdir(parents=True, exist_ok=True)
#     for disease_folder in [d for d in crop.iterdir() if d.is_dir()]:
#         for img in disease_folder.glob('*.jpg'):
#             # copy using a unique name to avoid clashes
#             dest = target / f"{disease_folder.name}_{img.name}"
#             if not dest.exists():
#                 shutil.copy(img, dest)

# print("Created crop-level dataset at:", OUT)


Created crop-level dataset at: data\crops_for_classification


In [6]:
# # installs (uncomment if needed)
# # !pip install tensorflow pillow matplotlib scikit-learn joblib opencv-python

# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# IMG_SIZE = (224,224)
# BATCH = 32

# # for crop classifier (Stage A)
# datagen_crop = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=20,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     zoom_range=0.1,
#     horizontal_flip=True,
#     validation_split=0.2
# )

# train_crop_gen = datagen_crop.flow_from_directory(
#     'data/crops_for_classification', target_size=IMG_SIZE,
#     batch_size=BATCH, subset='training', class_mode='categorical', shuffle=True)

# val_crop_gen = datagen_crop.flow_from_directory(
#     'data/crops_for_classification', target_size=IMG_SIZE,
#     batch_size=BATCH, subset='validation', class_mode='categorical', shuffle=False)

# print("Crop classes:", train_crop_gen.class_indices)


Found 10661 images belonging to 10 classes.
Found 2663 images belonging to 10 classes.
Crop classes: {'.ipynb_checkpoints': 0, 'corn': 1, 'data': 2, 'models': 3, 'potato': 4, 'rice': 5, 'saved_models': 6, 'src': 7, 'sugarcane': 8, 'wheat': 9}


In [7]:
# import joblib
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras import layers, Model
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# num_crops = train_crop_gen.num_classes
# base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
# x = layers.GlobalAveragePooling2D()(base.output)
# x = layers.Dense(256, activation='relu')(x)
# out = layers.Dense(num_crops, activation='softmax')(x)
# model_crop = Model(base.input, out)

# # freeze base for initial training
# for layer in base.layers:
#     layer.trainable = False

# model_crop.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# callbacks = [ 
#     ModelCheckpoint(str(MODEL_DIR/'crop_classifier.keras'), save_best_only=True, monitor='val_loss'),
#     EarlyStopping(patience=5, restore_best_weights=True)
# ]

# history = model_crop.fit(train_crop_gen, validation_data=val_crop_gen, epochs=5, callbacks=callbacks)

# # save mapping: class name -> index
# joblib.dump(train_crop_gen.class_indices, MODEL_DIR/'crop_label_map.joblib')
# print("Saved crop model and mapping.")


Epoch 1/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 654s 2s/step - accuracy: 0.9465 - loss: 0.2092 - val_accuracy: 0.9741 - val_loss: 0.0814
Epoch 2/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 639s 2s/step - accuracy: 0.9926 - loss: 0.0285 - val_accuracy: 0.9801 - val_loss: 0.0557
Epoch 3/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 684s 2s/step - accuracy: 0.9952 - loss: 0.0183 - val_accuracy: 0.9876 - val_loss: 0.0393
Epoch 4/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 637s 2s/step - accuracy: 0.9968 - loss: 0.0137 - val_accuracy: 0.9824 - val_loss: 0.0465
Epoch 5/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 636s 2s/step - accuracy: 0.9977 - loss: 0.0101 - val_accuracy: 0.9880 - val_loss: 0.0367
Saved crop model and mapping.


In [8]:
import os,shutil

# remove unwanted Jupyter hidden folder (this causes the error)
if os.path.exists('plant/.ipynb_checkpoints'):
    shutil.rmtree('plant/.ipynb_checkpoints')
    print("✅ Removed .ipynb_checkpoints folder if it existed.")

In [9]:
# # ==============================
# # 📘 Crop Disease Classification
# # ==============================

# import os
# from pathlib import Path
# import joblib
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras import layers, Model
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # -----------------------------
# # 1️⃣ Define constants & folders
# # -----------------------------
# BASE_DIR = Path(r"C:\Users\Kashish Kharb\plant")   # path where all crop folders exist
# MODEL_DIR = Path("models")
# MODEL_DIR.mkdir(exist_ok=True)  # make 'models' folder if it doesn't exist

# IMG_SIZE = (224, 224)
# BATCH = 16

# # -----------------------------
# # 2️⃣ Clean unwanted folders
# # -----------------------------
# # Remove any unwanted system folder (like .ipynb_checkpoints)
# for subdir in BASE_DIR.rglob('.ipynb_checkpoints'):
#     if subdir.is_dir():
#         print(f"Removing unwanted folder: {subdir}")
#         import shutil
#         shutil.rmtree(subdir)

# # -----------------------------
# # 3️⃣ Identify all crop folders
# # -----------------------------
# crops = [p.name for p in BASE_DIR.iterdir() if p.is_dir()]
# print("✅ Crops found:", crops)

# # -----------------------------
# # 4️⃣ Train model per crop type
# # -----------------------------
# for crop in crops:
#     print(f"\n🌾 Training model for crop: {crop}")

#     crop_path = BASE_DIR / crop
#     datagen = ImageDataGenerator(
#         rescale=1./255,
#         rotation_range=20,
#         width_shift_range=0.1,
#         height_shift_range=0.1,
#         zoom_range=0.1,
#         horizontal_flip=True,
#         validation_split=0.2
#     )

#     # Training and validation generators
#     train_gen = datagen.flow_from_directory(
#         crop_path,
#         target_size=IMG_SIZE,
#         batch_size=BATCH,
#         subset='training',
#         class_mode='categorical'
#     )
#     val_gen = datagen.flow_from_directory(
#         crop_path,
#         target_size=IMG_SIZE,
#         batch_size=BATCH,
#         subset='validation',
#         class_mode='categorical'
#     )

#     # Skip if less than 2 classes
#     if train_gen.num_classes < 2:
#         print(f"⚠️ Skipping {crop}: less than 2 disease classes.")
#         continue

#     # -----------------------------
#     # 5️⃣ Build model
#     # -----------------------------
#     base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#     for layer in base.layers:
#         layer.trainable = False

#     x = layers.GlobalAveragePooling2D()(base.output)
#     x = layers.Dense(256, activation='relu')(x)
#     out = layers.Dense(train_gen.num_classes, activation='softmax')(x)

#     model = Model(inputs=base.input, outputs=out)
#     model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

#     # -----------------------------
#     # 6️⃣ Callbacks
#     # -----------------------------
#     checkpoint_path = MODEL_DIR / f"disease_{crop}.keras"  # must end with .keras (new format)
#     callbacks = [
#         ModelCheckpoint(
#             # str(checkpoint_path),========
#             save_best_only=True,
#             monitor='val_loss'
#         ),
#         EarlyStopping(patience=5, restore_best_weights=True)
# ]
# # 7️⃣ Train model
# model.fit(train_gen, validation_data=val_gen, epochs=5, callbacks=callbacks)
# # 8️⃣ Save class mappings
# joblib.dump(train_gen.class_indices, MODEL_DIR / f'{crop}_class_map.joblib')
# print(f"✅ Saved model and class map for {crop}")

Removing unwanted folder: C:\Users\Kashish Kharb\plant\.ipynb_checkpoints
Removing unwanted folder: C:\Users\Kashish Kharb\plant\src\.ipynb_checkpoints
Removing unwanted folder: C:\Users\Kashish Kharb\plant\data\crops_for_classification\.ipynb_checkpoints
✅ Crops found: ['.ipynb_checkpoints', 'corn', 'data', 'models', 'potato', 'rice', 'saved_models', 'src', 'sugarcane', 'wheat']

🌾 Training model for crop: .ipynb_checkpoints
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
⚠️ Skipping .ipynb_checkpoints: less than 2 disease classes.

🌾 Training model for crop: corn
Found 3083 images belonging to 4 classes.
Found 769 images belonging to 4 classes.

🌾 Training model for crop: data
Found 10660 images belonging to 1 classes.
Found 2664 images belonging to 1 classes.
⚠️ Skipping data: less than 2 disease classes.

🌾 Training model for crop: models
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
⚠️ Skipping models: less than 2 disea

In [10]:
# # Save your trained model
# model.save("crop_disease_model.h5")
# print("✅ Model saved successfully!")

✅ Model saved successfully!


In [22]:
import json

with open("remedies.json", "r") as f:
    data = json.load(f)

print("Remedies loaded successfully:", data.keys())

Remedies loaded successfully: dict_keys(['corn_common_rust', 'corn_gray_leaf_spot', 'corn_healthy', 'corn_northern_leaf_blight', 'potato_early_blight', 'potato_healthy', 'potato_late_blight', 'rice_brown_spot', 'rice_healthy', 'rice_leaf_blast', 'rice_neck_blast', 'sugarcane_bacterial_blight', 'sugarcane_healthy', 'sugarcane_red_rot', 'wheat_brown_rust', 'wheat_healthy', 'wheat_yellow_rust'])


In [12]:
!pip install ipywidgets==8.1.2
!pip install jupyterlab_widgets

In [3]:
import io
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import json
from IPython.display import display
import ipywidgets as widgets
from pathlib import Path

# === Folder paths ===
TRAIN_DIR = Path("data/train")  # path where 'corn', 'rice', 'wheat' folders are
MODEL_DIR = Path("models")

# === Auto-load classes from folder names ===
if TRAIN_DIR.exists():
    classes = sorted(os.listdir(TRAIN_DIR))
    print(f"✅ Loaded class labels: {classes}")
else:
    print("⚠️ 'train' folder not found! Please check your path.")
    classes = []

# === Load model (if exists) ===
model_path = MODEL_DIR / "crop_classifier.keras"
if model_path.exists():
    model = load_model(model_path)
    print("✅ Model loaded successfully.")
else:
    model = None
    print("⚠️ No trained model found yet. Please train and save as 'models/crop_classifier.keras'.")

# === Load remedies ===
try:
    with open("remedies.json", "r") as f:
        remedies = json.load(f)
except FileNotFoundError:
    remedies = {}
    print("⚠️ remedies.json not found — remedies will be shown from default list.")

# === Default remedies if remedies.json missing ===
default_remedies = {
    "corn": "🌽 Remedy: Use resistant hybrids, rotate crops, and apply fungicides if needed.",
    "rice": "🌾 Remedy: Maintain field drainage, avoid excess nitrogen, use certified seeds.",
    "wheat": "🌿 Remedy: Remove infected leaves, apply fungicides, and use resistant varieties."
}

# === Widgets ===
uploader = widgets.FileUpload(accept='image/*', multiple=False)
predict_btn = widgets.Button(description="🔍 Predict Disease", button_style='success')
output = widgets.Output()

# Display UI
display(widgets.VBox([uploader, predict_btn, output]))

# === Prediction function ===
def process_upload(change=None):
    with output:
        output.clear_output()
        if not uploader.value:
            print("⚠️ Please upload an image first.")
            return

        # --- Fix uploader handling ---
        upload_data = uploader.value

        # Handle different formats (tuple or dict)
        if isinstance(upload_data, dict):
            v = next(iter(upload_data.values()))
        elif isinstance(upload_data, tuple) and len(upload_data) > 0 and isinstance(upload_data[0], dict):
            v = upload_data[0]
        else:
            print("⚠️ Could not read uploaded image.")
            return

        img_bytes = v['content']
        img = Image.open(io.BytesIO(img_bytes)).convert("RGB")

        display(img)

        # Preprocess
        img_resized = img.resize((224, 224))
        img_array = np.array(img_resized) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Predict
        pred = model.predict(img_array)
        pred_class_index = np.argmax(pred)
        confidence = np.max(pred) * 100
        predicted_label = classes[pred_class_index]

        # Remedy
        remedy = remedies.get(predicted_label, default_remedies.get(predicted_label.lower(), "No remedy found."))

        # Display results
        print("\n✅ Prediction Complete!")
        print(f"🌿 Predicted Crop: {predicted_label}")
        print(f"📊 Confidence: {confidence:.2f}%")
        print(f"💡 Suggested Remedy: {remedy}")

# === Connect button to function ===
predict_btn.on_click(process_upload)

print("📷 Step 1: Upload a leaf image\n🔍 Step 2: Click 'Predict Disease'")


✅ Loaded class labels: ['corn', 'potato', 'rice', 'sugarcane', 'wheat']
✅ Model loaded successfully.


📷 Step 1: Upload a leaf image
🔍 Step 2: Click 'Predict Disease'


In [24]:
import tensorflow as tf
from pathlib import Path

train_dir = Path("./data/train")

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32
)

print("🧾 Classes found:", dataset.class_names)


Found 13324 files belonging to 5 classes.
🧾 Classes found: ['corn', 'potato', 'rice', 'sugarcane', 'wheat']
